# Bagian yang masih sama dengan TP 3

## util.py

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

class IdMap:
    """
    Ingat kembali di kuliah, bahwa secara praktis, sebuah dokumen dan
    sebuah term akan direpresentasikan sebagai sebuah integer. Oleh
    karena itu, kita perlu maintain mapping antara string term (atau
    dokumen) ke integer yang bersesuaian, dan sebaliknya. Kelas IdMap ini
    akan melakukan hal tersebut.
    """

    def __init__(self):
        """
        Mapping dari string (term atau nama dokumen) ke id disimpan dalam
        python's dictionary; cukup efisien. Mapping sebaliknya disimpan dalam
        python's list.

        contoh:
            str_to_id["halo"] ---> 8
            str_to_id["/collection/dir0/gamma.txt"] ---> 54

            id_to_str[8] ---> "halo"
            id_to_str[54] ---> "/collection/dir0/gamma.txt"
        """
        self.str_to_id = {}
        self.id_to_str = []

    def __len__(self):
        """Mengembalikan banyaknya term (atau dokumen) yang disimpan di IdMap."""
        return len(self.id_to_str)

    def __get_str(self, i):
        """Mengembalikan string yang terasosiasi dengan index i."""
        # TODO
        return self.id_to_str[i]

    def __get_id(self, s):
        """
        Mengembalikan integer id i yang berkorespondensi dengan sebuah string s.
        Jika s tidak ada pada IdMap, lalu assign sebuah integer id baru dan kembalikan
        integer id baru tersebut.
        """
        # TODO
        if s not in self.id_to_str:
            self.id_to_str.append(s)
            idx = self.id_to_str.index(s)
            self.str_to_id[s] = idx
        return self.str_to_id[s]

    def __getitem__(self, key):
        """
        __getitem__(...) adalah special method di Python, yang mengizinkan sebuah
        collection class (seperti IdMap ini) mempunyai mekanisme akses atau
        modifikasi elemen dengan syntax [..] seperti pada list dan dictionary di Python.

        Silakan search informasi ini di Web search engine favorit Anda. Saya mendapatkan
        link berikut:

        https://stackoverflow.com/questions/43627405/understanding-getitem-method

        Jika key adalah integer, gunakan __get_str;
        jika key adalah string, gunakan __get_id
        """
        if type(key) is int:
            return self.__get_str(key)
        elif type(key) is str:
            return self.__get_id(key)
        else:
            raise TypeError

def sorted_merge_posts_and_tfs(posts_tfs1, posts_tfs2):
    """
    Menggabung (merge) dua lists of tuples (doc id, tf) dan mengembalikan
    hasil penggabungan keduanya (TF perlu diakumulasikan untuk semua tuple
    dengn doc id yang sama), dengan aturan berikut:

    contoh: posts_tfs1 = [(1, 34), (3, 2), (4, 23)]
            posts_tfs2 = [(1, 11), (2, 4), (4, 3 ), (6, 13)]

            return   [(1, 34+11), (2, 4), (3, 2), (4, 23+3), (6, 13)]
                   = [(1, 45), (2, 4), (3, 2), (4, 26), (6, 13)]

    Parameters
    ----------
    list1: List[(Comparable, int)]
    list2: List[(Comparable, int]
        Dua buah sorted list of tuples yang akan di-merge.

    Returns
    -------
    List[(Comparablem, int)]
        Penggabungan yang sudah terurut
    """
    # TODO
    sorted_lst = []
    p1, p2 = 0, 0
    while p1 < len(posts_tfs1) and p2 < len(posts_tfs2):
        if (posts_tfs1[p1][0] == posts_tfs2[p2][0]):
            same_term = posts_tfs1[p1][0]
            sum_tf = posts_tfs1[p1][1] + posts_tfs2[p2][1]
            sorted_lst.append((same_term, sum_tf))
            p1 += 1
            p2 += 1
        elif posts_tfs1[p1] < posts_tfs2[p2]:
            sorted_lst.append(posts_tfs1[p1])
            p1 += 1
        else:
            sorted_lst.append(posts_tfs2[p2])
            p2 += 1
    else:
        if p1 < len(posts_tfs1):
            sorted_lst.append(posts_tfs1[p1])
            p1 += 1
        elif p2 < len(posts_tfs2):
            sorted_lst.append(posts_tfs2[p2])
            p2 += 2

    return sorted_lst

def text_preprocess(string):
    "Mengembalikan hasil preprocess dari suatu teks string."
    # Normalization
    res = string.lower().strip()
    res = re.sub("\d", "", res) # Menghilangkan angka
    res = re.sub("\s+", " ", res) # Menghilangkan spasi berlebih
    res = re.sub("[^\w\s]", " ", res) # Menghilangkan tanda baca

    words = word_tokenize(res)

    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]

    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]

    return stemmed

def test(output, expected):
    """ simple function for testing """
    return "PASSED" if output == expected else "FAILED"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## compression.py

In [3]:
import array

class VBEPostings:
    """ 
    Berbeda dengan StandardPostings, dimana untuk suatu postings list,
    yang disimpan di disk adalah sequence of integers asli dari postings
    list tersebut apa adanya.

    Pada VBEPostings, kali ini, yang disimpan adalah gap-nya, kecuali
    posting yang pertama. Barulah setelah itu di-encode dengan Variable-Byte
    Enconding algorithm ke bytestream.

    Contoh:
    postings list [34, 67, 89, 454] akan diubah dulu menjadi gap-based,
    yaitu [34, 33, 22, 365]. Barulah setelah itu di-encode dengan algoritma
    compression Variable-Byte Encoding, dan kemudian diubah ke bytesream.

    ASUMSI: postings_list untuk sebuah term MUAT di memori!

    """

    @staticmethod
    def vb_encode_number(number):
        """
        Encodes a number using Variable-Byte Encoding
        Lihat buku teks kita!
        """
        bytes = []
        while True:
            bytes.insert(0, number % 128) # prepend ke depan
            if number < 128:
                break
            number = number // 128
        bytes[-1] += 128 # bit awal pada byte terakhir diganti 1
        return array.array('B', bytes).tobytes()

    @staticmethod
    def vb_encode(list_of_numbers):
        """ 
        Melakukan encoding (tentunya dengan compression) terhadap
        list of numbers, dengan Variable-Byte Encoding
        """
        bytes = []
        for number in list_of_numbers:
            bytes.append(VBEPostings.vb_encode_number(number))
        return b"".join(bytes)

    @staticmethod
    def encode(postings_list):
        """
        Encode postings_list menjadi stream of bytes (dengan Variable-Byte
        Encoding). JANGAN LUPA diubah dulu ke gap-based list, sebelum
        di-encode dan diubah ke bytearray.

        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)

        Returns
        -------
        bytes
            bytearray yang merepresentasikan urutan integer di postings_list
        """
        # TODO
        gap_list = [postings_list[0]] + [postings_list[i+1] - postings_list[i] for i in range(len(postings_list) - 1)]
        return VBEPostings.vb_encode(gap_list)

    @staticmethod
    def encode_tf(tf_list):
        """
        Encode list of term frequencies menjadi stream of bytes

        Parameters
        ----------
        tf_list: List[int]
            List of term frequencies

        Returns
        -------
        bytes
            bytearray yang merepresentasikan nilai raw TF kemunculan term di setiap
            dokumen pada list of postings
        """
        return VBEPostings.vb_encode(tf_list)

    @staticmethod
    def vb_decode(encoded_bytestream):
        """
        Decoding sebuah bytestream yang sebelumnya di-encode dengan
        variable-byte encoding.
        """
        # TODO
        numbers = []
        n = 0
        for i in range(len(encoded_bytestream)):
            if encoded_bytestream[i] < 128:
                n = 128 * n + encoded_bytestream[i]
            else:
                n = 128 * n + (encoded_bytestream[i] - 128)
                numbers.append(n)
                n = 0
        return numbers

    @staticmethod
    def decode(encoded_postings_list):
        """
        Decodes postings_list dari sebuah stream of bytes. JANGAN LUPA
        bytestream yang di-decode dari encoded_postings_list masih berupa
        gap-based list.

        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray merepresentasikan encoded postings list sebagai keluaran
            dari static method encode di atas.

        Returns
        -------
        List[int]
            list of docIDs yang merupakan hasil decoding dari encoded_postings_list
        """
        # TODO
        gap_list = VBEPostings.vb_decode(encoded_postings_list)
        return [sum(gap_list[:i+1]) for i in range(len(gap_list))]

    @staticmethod
    def decode_tf(encoded_tf_list):
        """
        Decodes list of term frequencies dari sebuah stream of bytes

        Parameters
        ----------
        encoded_tf_list: bytes
            bytearray merepresentasikan encoded term frequencies list sebagai keluaran
            dari static method encode_tf di atas.

        Returns
        -------
        List[int]
            List of term frequencies yang merupakan hasil decoding dari encoded_tf_list
        """
        return VBEPostings.vb_decode(encoded_tf_list)

##index.py

In [4]:
import pickle
import os

class InvertedIndex:
    """
    Class yang mengimplementasikan bagaimana caranya scan atau membaca secara
    efisien Inverted Index yang disimpan di sebuah file; dan juga menyediakan
    mekanisme untuk menulis Inverted Index ke file (storage) saat melakukan indexing.

    Attributes
    ----------
    postings_dict: Dictionary mapping:

            termID -> (start_position_in_index_file,
                       number_of_postings_in_list,
                       length_in_bytes_of_postings_list,
                       length_in_bytes_of_tf_list)

        postings_dict adalah konsep "Dictionary" yang merupakan bagian dari
        Inverted Index. postings_dict ini diasumsikan dapat dimuat semuanya
        di memori.

        Seperti namanya, "Dictionary" diimplementasikan sebagai python's Dictionary
        yang memetakan term ID (integer) ke 4-tuple:
           1. start_position_in_index_file : (dalam satuan bytes) posisi dimana
              postings yang bersesuaian berada di file (storage). Kita bisa
              menggunakan operasi "seek" untuk mencapainya.
           2. number_of_postings_in_list : berapa banyak docID yang ada pada
              postings (Document Frequency)
           3. length_in_bytes_of_postings_list : panjang postings list dalam
              satuan byte.
           4. length_in_bytes_of_tf_list : panjang list of term frequencies dari
              postings list terkait dalam satuan byte

    terms: List[int]
        List of terms IDs, untuk mengingat urutan terms yang dimasukan ke
        dalam Inverted Index.

    """
    def __init__(self, index_name, postings_encoding, directory=''):
        """
        Parameters
        ----------
        index_name (str): Nama yang digunakan untuk menyimpan files yang berisi index
        postings_encoding : Lihat di compression.py, kandidatnya adalah StandardPostings,
                        GapBasedPostings, dsb.
        directory (str): directory dimana file index berada
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []             # Untuk keep track urutan term yang dimasukkan ke index
        self.doc_length = {}        # key: doc ID (int), value: document length (number of tokens)
        self.average_doc_length = 0 # Ini nantinya akan berguna untuk normalisasi Score terhadap panjang
                                    # dokumen saat menghitung score dengan TF-IDF atau BM25

    def __enter__(self):
        """
        Memuat semua metadata ketika memasuki context.
        Metadata:
            1. Dictionary ---> postings_dict
            2. iterator untuk List yang berisi urutan term yang masuk ke
                index saat konstruksi. ---> term_iter
            3. doc_length, sebuah python's dictionary yang berisi key = doc id, dan
                value berupa banyaknya token dalam dokumen tersebut (panjang dokumen).
                Berguna untuk normalisasi panjang saat menggunakan TF-IDF atau BM25
                scoring regime; berguna untuk untuk mengetahui nilai N saat hitung IDF,
                dimana N adalah banyaknya dokumen di koleksi

        Metadata disimpan ke file dengan bantuan library "pickle"

        Perlu memahani juga special method __enter__(..) pada Python dan juga
        konsep Context Manager di Python. Silakan pelajari link berikut:

        https://docs.python.org/3/reference/datamodel.html#object.__enter__
        """
        # Membuka index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Kita muat postings dict dan terms iterator dari file metadata
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms, self.doc_length = pickle.load(f)
            self.term_iter = self.terms.__iter__()

        self.average_doc_length = sum(self.doc_length.values()) / len(self.doc_length)
        
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        """Menutup index_file dan menyimpan postings_dict dan terms ketika keluar context.
            Selain itu, juga menghitung rata-rata panjang dokumen."""
        # Menutup index file
        self.index_file.close()
        # Menyimpan metadata (postings dict dan terms) ke file metadata dengan bantuan pickle
        with open(self.metadata_file_path, 'wb') as f:
            pickle.dump([self.postings_dict, self.terms, self.doc_length], f)


class InvertedIndexReader(InvertedIndex):
    """
    Class yang mengimplementasikan bagaimana caranya scan atau membaca secara
    efisien Inverted Index yang disimpan di sebuah file.
    """
    def __iter__(self):
        return self

    def reset(self):
        """
        Kembalikan file pointer ke awal, dan kembalikan pointer iterator
        term ke awal
        """
        self.index_file.seek(0)
        self.term_iter = self.terms.__iter__() # reset term iterator

    def __next__(self): 
        """
        Class InvertedIndexReader juga bersifat iterable (mempunyai iterator).
        Silakan pelajari:
        https://stackoverflow.com/questions/19151/how-to-build-a-basic-iterator

        Ketika instance dari kelas InvertedIndexReader ini digunakan
        sebagai iterator pada sebuah loop scheme, special method __next__(...)
        bertugas untuk mengembalikan pasangan (term, postings_list, tf_list) berikutnya
        pada inverted index.

        PERHATIAN! method ini harus mengembalikan sebagian kecil data dari
        file index yang besar. Mengapa hanya sebagian kecil? karena agar muat
        diproses di memori. JANGAN MEMUAT SEMUA INDEX DI MEMORI!
        """
        curr_term = next(self.term_iter)
        # pos, number_of_postings, len_in_bytes_of_postings, len_in_bytes_of_tf = self.postings_dict[curr_term]
        # postings_list = self.postings_encoding.decode(self.index_file.read(len_in_bytes_of_postings))
        # tf_list = self.postings_encoding.decode_tf(self.index_file.read(len_in_bytes_of_tf))
        postings_list, tf_list = self.get_postings_list(curr_term)
        return (curr_term, postings_list, tf_list)

    def get_postings_list(self, term):
        """
        Kembalikan sebuah postings list (list of docIDs) beserta list
        of term frequencies terkait untuk sebuah term (disimpan dalam
        bentuk tuple (postings_list, tf_list)).

        PERHATIAN! method tidak boleh iterasi di keseluruhan index
        dari awal hingga akhir. Method ini harus langsung loncat ke posisi
        byte tertentu pada file (index file) dimana postings list (dan juga
        list of TF) dari term disimpan.
        """
        # TODO
        start, _, len_in_bytes_of_postings, \
            len_in_bytes_of_tf = self.postings_dict[term]

        self.index_file.seek(start)
        encoded_postings_list = self.index_file.read(len_in_bytes_of_postings)
        encoded_tf_list = self.index_file.read(len_in_bytes_of_tf)
        decoded_postings_list = self.postings_encoding.decode(encoded_postings_list)
        decoded_tf_list = self.postings_encoding.decode_tf(encoded_tf_list)

        return (decoded_postings_list, decoded_tf_list)


class InvertedIndexWriter(InvertedIndex):
    """
    Class yang mengimplementasikan bagaimana caranya menulis secara
    efisien Inverted Index yang disimpan di sebuah file.
    """
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')
        return self

    def append(self, term, postings_list, tf_list):
        """
        Menambahkan (append) sebuah term, postings_list, dan juga TF list 
        yang terasosiasi ke posisi akhir index file.

        Method ini melakukan 4 hal:
        1. Encode postings_list menggunakan self.postings_encoding (method encode),
        2. Encode tf_list menggunakan self.postings_encoding (method encode_tf),
        3. Menyimpan metadata dalam bentuk self.terms, self.postings_dict, dan self.doc_length.
           Ingat kembali bahwa self.postings_dict memetakan sebuah termID ke
           sebuah 4-tuple: - start_position_in_index_file
                           - number_of_postings_in_list
                           - length_in_bytes_of_postings_list
                           - length_in_bytes_of_tf_list
        4. Menambahkan (append) bystream dari postings_list yang sudah di-encode dan
           tf_list yang sudah di-encode ke posisi akhir index file di harddisk.

        Jangan lupa update self.terms dan self.doc_length juga ya!

        SEARCH ON YOUR FAVORITE SEARCH ENGINE:
        - Anda mungkin mau membaca tentang Python I/O
          https://docs.python.org/3/tutorial/inputoutput.html
          Di link ini juga bisa kita pelajari bagaimana menambahkan informasi
          ke bagian akhir file.
        - Beberapa method dari object file yang mungkin berguna seperti seek(...)
          dan tell()

        Parameters
        ----------
        term:
            term atau termID yang merupakan unique identifier dari sebuah term
        postings_list: List[Int]
            List of docIDs dimana term muncul
        tf_list: List[Int]
            List of term frequencies
        """
        # TODO
        encoded_postings_list = self.postings_encoding.encode(postings_list)
        encoded_tf = self.postings_encoding.encode_tf(tf_list)

        for doc_id, tf in zip(postings_list, tf_list):
            if doc_id not in self.doc_length:
                self.doc_length[doc_id] = tf
            else:
                self.doc_length[doc_id] += tf
        
        self.postings_dict[term] = (self.index_file.tell(),
                                    len(postings_list),
                                    len(encoded_postings_list),
                                    len(encoded_tf))
        
        self.terms.append(term)

        self.index_file.write(encoded_postings_list)
        self.index_file.write(encoded_tf)

## bsbi.py

In [5]:
import os
import pickle
import contextlib
import glob
import heapq
import time
import math
from tqdm import tqdm

class BSBIIndex:
    """
    Attributes
    ----------
    term_id_map(IdMap): Untuk mapping terms ke termIDs
    doc_id_map(IdMap): Untuk mapping relative paths dari dokumen (misal,
                    /collection/0/gamma.txt) to docIDs
    data_dir(str): Path ke data
    output_dir(str): Path ke output index files
    postings_encoding: Lihat di compression.py, kandidatnya adalah StandardPostings,
                    VBEPostings, dsb.
    index_name(str): Nama dari file yang berisi inverted index
    """
    def __init__(self, data_dir, output_dir, postings_encoding, index_name = "main_index"):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Untuk menyimpan nama-nama file dari semua intermediate inverted index
        self.intermediate_indices = []

    def save(self):
        """Menyimpan doc_id_map and term_id_map ke output directory via pickle"""

        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pickle.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pickle.dump(self.doc_id_map, f)

    def load(self):
        """Memuat doc_id_map and term_id_map dari output directory"""

        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pickle.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pickle.load(f)

    def parse_block(self, block_dir_relative):
        """
        Lakukan parsing terhadap text file sehingga menjadi sequence of
        <termID, docID> pairs.

        Gunakan tools available untuk Stemming Bahasa Inggris

        JANGAN LUPA BUANG STOPWORDS!

        Untuk "sentence segmentation" dan "tokenization", bisa menggunakan
        regex atau boleh juga menggunakan tools lain yang berbasis machine
        learning.

        Parameters
        ----------
        block_dir_relative : str
            Relative Path ke directory yang mengandung text files untuk sebuah block.

            CATAT bahwa satu folder di collection dianggap merepresentasikan satu block.
            Konsep block di soal tugas ini berbeda dengan konsep block yang terkait
            dengan operating systems.

        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
            Mengembalikan semua pasangan <termID, docID> dari sebuah block (dalam hal
            ini sebuah sub-direktori di dalam folder collection)

        Harus menggunakan self.term_id_map dan self.doc_id_map untuk mendapatkan
        termIDs dan docIDs. Dua variable ini harus 'persist' untuk semua pemanggilan
        parse_block(...).
        """
        # TODO
        dir_path = os.path.join(self.data_dir, block_dir_relative)
        td_pairs = []

        for filename in os.listdir(dir_path):
            with open(os.path.join(dir_path, filename), 'r') as f:
                string = f.read()
                tokens = text_preprocess(string)

                doc_id = self.doc_id_map[f.name]
                for token in tokens:
                    term_id = self.term_id_map[token]
                    td_pairs.append((term_id, doc_id))

        return td_pairs

    def invert_write(self, td_pairs, index):
        """
        Melakukan inversion td_pairs (list of <termID, docID> pairs) dan
        menyimpan mereka ke index. Disini diterapkan konsep BSBI dimana 
        hanya di-mantain satu dictionary besar untuk keseluruhan block.
        Namun dalam teknik penyimpanannya digunakan srategi dari SPIMI
        yaitu penggunaan struktur data hashtable (dalam Python bisa
        berupa Dictionary)

        ASUMSI: td_pairs CUKUP di memori

        Di Tugas Pemrograman 1, kita hanya menambahkan term dan
        juga list of sorted Doc IDs. Sekarang di Tugas Pemrograman 2,
        kita juga perlu tambahkan list of TF.

        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index pada disk (file) yang terkait dengan suatu "block"
        """
        # TODO
        term_dict = {}
        for term_id, doc_id in td_pairs:
            if term_id not in term_dict:
                term_dict[term_id] = dict()
            if doc_id not in term_dict[term_id]:
                term_dict[term_id][doc_id] = 0
            term_dict[term_id][doc_id] += 1
        for term_id in sorted(term_dict.keys()):
            doc_id_freq_pairs = sorted(term_dict[term_id].items())
            separated_pairs = list(zip(*doc_id_freq_pairs))
            postings_list = list(separated_pairs[0])
            tf_list = list(separated_pairs[1])
            index.append(term_id, postings_list, tf_list)

    def merge(self, indices, merged_index):
        """
        Lakukan merging ke semua intermediate inverted indices menjadi
        sebuah single index.

        Ini adalah bagian yang melakukan EXTERNAL MERGE SORT

        Gunakan fungsi orted_merge_posts_and_tfs(..) di modul util

        Parameters
        ----------
        indices: List[InvertedIndexReader]
            A list of intermediate InvertedIndexReader objects, masing-masing
            merepresentasikan sebuah intermediate inveted index yang iterable
            di sebuah block.

        merged_index: InvertedIndexWriter
            Instance InvertedIndexWriter object yang merupakan hasil merging dari
            semua intermediate InvertedIndexWriter objects.
        """
        # kode berikut mengasumsikan minimal ada 1 term
        merged_iter = heapq.merge(*indices, key = lambda x: x[0])
        curr, postings, tf_list = next(merged_iter) # first item
        for t, postings_, tf_list_ in merged_iter: # from the second item
            if t == curr:
                zip_p_tf = sorted_merge_posts_and_tfs(list(zip(postings, tf_list)), \
                                                      list(zip(postings_, tf_list_)))
                postings = [doc_id for (doc_id, _) in zip_p_tf]
                tf_list = [tf for (_, tf) in zip_p_tf]
            else:
                merged_index.append(curr, postings, tf_list)
                curr, postings, tf_list = t, postings_, tf_list_
        merged_index.append(curr, postings, tf_list)

    def retrieve_tfidf(self, query, k = 10):
        """
        Melakukan Ranked Retrieval dengan skema TaaT (Term-at-a-Time).
        Method akan mengembalikan top-K retrieval results.

        w(t, D) = (1 + log tf(t, D))       jika tf(t, D) > 0
                = 0                        jika sebaliknya

        w(t, Q) = IDF = log (N / df(t))

        Score = untuk setiap term di query, akumulasikan w(t, Q) * w(t, D).
                (tidak perlu dinormalisasi dengan panjang dokumen)

        catatan: 
            1. informasi DF(t) ada di dictionary postings_dict pada merged index
            2. informasi TF(t, D) ada di tf_li
            3. informasi N bisa didapat dari doc_length pada merged index, len(doc_length)

        Parameters
        ----------
        query: str
            Query tokens yang dipisahkan oleh spasi

            contoh: Query "universitas indonesia depok" artinya ada
            tiga terms: universitas, indonesia, dan depok

        Result
        ------
        List[(int, str)]
            List of tuple: elemen pertama adalah score similarity, dan yang
            kedua adalah nama dokumen.
            Daftar Top-K dokumen terurut mengecil BERDASARKAN SKOR.

        JANGAN LEMPAR ERROR/EXCEPTION untuk terms yang TIDAK ADA di collection.

        """
        # TODO
        tokenized = text_preprocess(query)

        self.load()
        postings_tf_lists = []
        n = 0

        with InvertedIndexReader(self.index_name, self.postings_encoding, directory=self.output_dir) as inv_reader:
            for token in tokenized:
                if token not in self.term_id_map:
                    continue
                postings_tf_lists.append(inv_reader.get_postings_list(self.term_id_map[token]))
            n = len(inv_reader.doc_length)

        # TaaT
        result = {}
        for i in range(len(postings_tf_lists)):
            df = len(postings_tf_lists[i][0])
            wtq = math.log10(n/df)

            for j in range(df):
                tf = postings_tf_lists[i][1][j]
                wtd = (1 + math.log10(tf)) if tf > 0 else 0
                score = wtd * wtq
                doc_name = self.doc_id_map[postings_tf_lists[i][0][j]]
                if doc_name in result:
                    result[doc_name] += score
                else:
                    result[doc_name] = score
        
        result = result.items()
        result = sorted(result, key=lambda x: x[1], reverse=True)[:k]
        return result

    def retrieve_bm25(self, query, k, k1 = 1.5, b = 0.75):
        """
        Melakukan Ranked Retrieval dengan skema TaaT (Term-at-a-Time).
        Method akan mengembalikan top-K retrieval results.

        w(t, D) = ((k1 + 1) * tf(t, D)) / ((k1 * ((1-b) + (b*dl/avdl))) + tf(t, D))

        w(t, Q) = IDF = log (N / df(t))

        Score = untuk setiap term di query, akumulasikan w(t, Q) * w(t, D).
                (tidak perlu dinormalisasi dengan panjang dokumen)

        catatan: 
            1. informasi DF(t) ada di dictionary postings_dict pada merged index
            2. informasi TF(t, D) ada di tf_li
            3. informasi N bisa didapat dari doc_length pada merged index, len(doc_length)

        Parameters
        ----------
        query: str
            Query tokens yang dipisahkan oleh spasi

            contoh: Query "universitas indonesia depok" artinya ada
            tiga terms: universitas, indonesia, dan depok

        Result
        ------
        List[(int, str)]
            List of tuple: elemen pertama adalah score similarity, dan yang
            kedua adalah nama dokumen.
            Daftar Top-K dokumen terurut mengecil BERDASARKAN SKOR.

        JANGAN LEMPAR ERROR/EXCEPTION untuk terms yang TIDAK ADA di collection.

        """
        # TODO
        tokenized = text_preprocess(query)

        self.load()
        postings_tf_lists = []
        n = 0
        result = {}

        with InvertedIndexReader(self.index_name, self.postings_encoding, directory=self.output_dir) as inv_reader:
            for token in tokenized:
                if token not in self.term_id_map:
                    continue
                postings_tf_lists.append(inv_reader.get_postings_list(self.term_id_map[token]))
            n = len(inv_reader.doc_length)

            # TaaT
            for i in range(len(postings_tf_lists)):
                df = len(postings_tf_lists[i][0])
                wtq = math.log10(n/df)

                for j in range(df):
                    tf = postings_tf_lists[i][1][j]
                    doc_id = postings_tf_lists[i][0][j]
                    dl = inv_reader.doc_length[doc_id]
                    avdl = inv_reader.average_doc_length
                    wtd = ((k1 + 1) * tf) / ((k1 * ((1-b) + (b*dl/avdl))) + tf)
                    score = wtd * wtq
                    doc_name = self.doc_id_map[doc_id]
                    if doc_name in result:
                        result[doc_name] += score
                    else:
                        result[doc_name] = score
        
        result = result.items()
        result = sorted(result, key=lambda x: x[1], reverse=True)[:k]
        return result

    def index(self):
        """
        Base indexing code
        BAGIAN UTAMA untuk melakukan Indexing dengan skema BSBI (blocked-sort
        based indexing)

        Method ini scan terhadap semua data di collection, memanggil parse_block
        untuk parsing dokumen dan memanggil invert_write yang melakukan inversion
        di setiap block dan menyimpannya ke index yang baru.
        """
        # loop untuk setiap sub-directory di dalam folder collection (setiap block)
        for block_dir_relative in tqdm(sorted(next(os.walk(self.data_dir))[1])):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'intermediate_index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, self.postings_encoding, directory = self.output_dir) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
    
        self.save()

        with InvertedIndexWriter(self.index_name, self.postings_encoding, directory = self.output_dir) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(InvertedIndexReader(index_id, self.postings_encoding, directory=self.output_dir))
                               for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

In [6]:
!mkdir /content/drive/MyDrive/IR/index

mkdir: cannot create directory ‘/content/drive/MyDrive/IR/index’: File exists


In [7]:
BSBI_instance = BSBIIndex(data_dir = '/content/drive/MyDrive/IR/collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = '/content/drive/MyDrive/IR/index')
BSBI_instance.index() # memulai indexing!

100%|██████████| 11/11 [01:42<00:00,  9.36s/it]


# Bagian yang berbeda dari TP 3

## experiment.py

### Inisialisasi model BioClinicalBERT

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 182 kB 74.0 MB/s 
     |████████████████████████████████| 7.6 MB 63.0 MB/s 


In [9]:
from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
bert_model = TFBertModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", \
                                         output_attentions=True, \
                                         from_pt = True)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

### Menambahkan layer classifier dari output BioClinicalBERT

In [10]:
import tensorflow as tf

from transformers import TFBertModel
from keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

class MedlineModel(tf.keras.Model):

    def __init__(self, num_class=3,
                 model_name="emilyalsentzer/Bio_ClinicalBERT", dropout_prob=0.1):
        super().__init__(name="Medline_Model")
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.dropout = Dropout(dropout_prob)
        self.dense_classifier = Dense(num_class,name="dense_classifier")

    def call(self, inputs, **kwargs):
        # get pooler output for CLS embedding
        trained_bert = self.bert(inputs, **kwargs)
        cls_embed = trained_bert.pooler_output
        
        sequence_output = self.dropout(cls_embed,
                                       training=kwargs.get("training", False))
        output_logits = self.dense_classifier(sequence_output)

        return output_logits


In [11]:
model = MedlineModel()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [12]:
# Model format h5 tidak menyimpan ukuran input, maka itu kita perlu inisialisasi
# ukuran input yang sesuai dengan proses train sebelum memprediksi
model_shape = tokenizer('dummy', 'text', padding=True, truncation=True, max_length=300, return_tensors='tf')
model(model_shape)
model.load_weights('/content/drive/MyDrive/IR/model/weight_bioclinicalbert_epoch_11.h5')

In [13]:
def encode_texts(documents, max_length = None):
    return tokenizer(documents, padding='max_length', max_length=max_length,
                     truncation=True)

In [14]:
import numpy as np

def create_pred(encoded_inputs):
    """Membuat semua bagian hasil tokenizer menjadi ndarray numpy"""
    ids = np.array(encoded_inputs['input_ids'])
    tid = np.array(encoded_inputs['token_type_ids'])
    ams = np.array(encoded_inputs['attention_mask'])

    return {"input_ids": ids, "token_type_ids": tid,  "attention_mask": ams}

In [15]:
import ntpath

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

### Menyiapkan metrik evaluasi untuk reranking (sama seperti TP3)

In [16]:
import re
import math
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

######## >>>>> 3 IR metrics: RBP p = 0.8, DCG, dan AP

def rbp(ranking, p = 0.8):
  """ menghitung search effectiveness metric score dengan 
      Rank Biased Precision (RBP)

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score RBP
  """
  score = 0.
  for i in range(1, len(ranking) + 1):
    pos = i - 1
    score += ranking[pos] * (p ** (i - 1))
  return (1 - p) * score

def dcg(ranking):
  """ menghitung search effectiveness metric score dengan 
      Discounted Cumulative Gain

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score DCG
  """
  # TODO
  score = 0
  for i in range(len(ranking)):
    score += ranking[i]/math.log2(i+2)
  return score

def ap(ranking):
  """ menghitung search effectiveness metric score dengan 
      Average Precision

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score AP
  """
  # TODO
  def prec_k(k):
    score = 0
    for i in range(k):
      score += ranking[i]/(k)
    return score
    
  R = sum(ranking) if sum(ranking) > 0 else 1

  score = 0
  for k in range(len(ranking)):
    score += (prec_k(k+1)*ranking[k])/R
  return score

######## >>>>> memuat qrels

def load_qrels(qrel_file = "qrels.txt", max_q_id = 30, max_doc_id = 1033):
  """ memuat query relevance judgment (qrels) 
      dalam format dictionary of dictionary
      qrels[query id][document id]

      dimana, misal, qrels["Q3"][12] = 1 artinya Doc 12
      relevan dengan Q3; dan qrels["Q3"][10] = 0 artinya
      Doc 10 tidak relevan dengan Q3.

  """
  qrels = {"Q" + str(i) : {i:0 for i in range(1, max_doc_id + 1)} \
                 for i in range(1, max_q_id + 1)}
  with open(qrel_file) as file:
    for line in file:
      parts = line.strip().split()
      qid = parts[0]
      did = int(parts[1])
      qrels[qid][did] = 1
  return qrels

### Metode evaluasi yang berbeda dari TP 3

In [22]:
######## >>>>> EVALUASI !
from functools import reduce

def eval(qrels, query_file = "queries.txt", k = 1000, reranking = True,
         MAX_LENGTH = 300, bm25 = True):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-k documents
  """
  BSBI_instance = BSBIIndex(data_dir = '/content/drive/MyDrive/IR/collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = '/content/drive/MyDrive/IR/index')

  dict_qd = dict()

  dict_index = dict()

  detokenizer = TreebankWordDetokenizer()

  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for qline in file:
      parts = qline.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])
      qd_pred = []
      doc_idx = []

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []

      if bm25:
        retrieved = BSBI_instance.retrieve_bm25(query, k = k, k1 = 2, b = 0.75)
      else:
        retrieved = BSBI_instance.retrieve_tfidf(query, k = k)
      for (doc, score) in retrieved:
          did = int(re.search(r'(.*)\.txt', path_leaf(doc)).group(1))
          doc_idx.append(did)
          if reranking:
            # simpan konten dokumen untuk bahan prediksi model
            with open(doc, encoding="utf-8") as collection_file:
              tokenized_doc = word_tokenize(collection_file.read())
              str_doc = detokenizer.detokenize(tokenized_doc)
              qd_pred.append((query, str_doc))
          ranking.append(qrels[qid][did])
      dict_qd[qid] = qd_pred
      dict_index[qid] = doc_idx

      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))
  print(f"Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = {k}) terhadap 30 queries")
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))
  

  if reranking:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for qid in dict_qd.keys():
      qd_pred = dict_qd[qid]
      doc_idx = dict_index[qid]

      encoded_inputs_pred = encode_texts(qd_pred, max_length = MAX_LENGTH)
      x = create_pred(encoded_inputs_pred)

      # Hasil prediksi adalah 2d array berukuran banyak_data row
      # dan banyak_kelas kolom. Karena modelnya dilatih untuk menghasilkan
      # tiga kelas, maka nanti akan ada tiga kolom
      predicted = model.predict(x)

      # Skor cukup dihitung dengan ambil index yang paling besar
      # elemennya (cari argmax).
      # misal pred = [2.33 9.232 1.2], maka score-nya adalah 1
      scores = [np.argmax(pred) for pred in predicted]

      doc_id_scores = [x for x in zip([did for did in doc_idx], scores)]
      sorted_scores = sorted(doc_id_scores, key = lambda tup: tup[1], reverse = True)

      ranking = [qrels[qid][did] for (did, _) in sorted_scores]

      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))

    print("Hasil evaluasi reranking")
    print("RBP score =", sum(rbp_scores) / len(rbp_scores))
    print("DCG score =", sum(dcg_scores) / len(dcg_scores))
    print("AP score  =", sum(ap_scores) / len(ap_scores))

### Hasil evaluasi

#### Top 5 documents

In [23]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k=5)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 5) terhadap 30 queries
RBP score = 0.3575786666666666
DCG score = 1.583826249345555
AP score  = 0.6806944444444445
1/1 [==============================] - 0s 128ms/step
Hasil evaluasi reranking
RBP score = 0.3575786666666666
DCG score = 1.583826249345555
AP score  = 0.6806944444444445


#### Top 10 documents

In [24]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k=10)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 10) terhadap 30 queries
RBP score = 0.4213984972800001
DCG score = 2.0179717759990576
AP score  = 0.6431168430335098
1/1 [==============================] - 0s 229ms/step
Hasil evaluasi reranking
RBP score = 0.4213984972800001
DCG score = 2.0179717759990576
AP score  = 0.6431168430335098


#### Top 50 documents

In [25]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k = 50)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 50) terhadap 30 queries
RBP score = 0.4416710196150325
DCG score = 2.8012324128494015
AP score  = 0.5162565797421899
2/2 [==============================] - 1s 395ms/step
Hasil evaluasi reranking
RBP score = 0.4416710196150325
DCG score = 2.8012324128494015
AP score  = 0.5162565797421899


#### Top 100 documents

In [26]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k = 100)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 100) terhadap 30 queries
RBP score = 0.44167130187927167
DCG score = 2.963526452253798
AP score  = 0.4824332968670388
4/4 [==============================] - 2s 483ms/step
Hasil evaluasi reranking
RBP score = 0.44167130187927167
DCG score = 2.963526452253798
AP score  = 0.4824332968670388


#### Top 500 documents

In [27]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k = 500)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 500) terhadap 30 queries
RBP score = 0.4416713018796457
DCG score = 2.991795411969586
AP score  = 0.4769026681312379
5/5 [==============================] - 3s 507ms/step
Hasil evaluasi reranking
RBP score = 0.4416713018796457
DCG score = 2.991795411969586
AP score  = 0.4769026681312379


#### Top 1000 documents

In [28]:
qrels = load_qrels(qrel_file='/content/drive/MyDrive/IR/qrels.txt')
assert qrels["Q1"][166] == 1, "qrels salah"
assert qrels["Q1"][300] == 0, "qrels salah"

eval(qrels, query_file='/content/drive/MyDrive/IR/queries.txt', k = 1000)

Hasil evaluasi BM25 (k1 = 2, b = 0.75, k = 1000) terhadap 30 queries
RBP score = 0.4416713018796457
DCG score = 2.991795411969586
AP score  = 0.4769026681312379
5/5 [==============================] - 3s 514ms/step
Hasil evaluasi reranking
RBP score = 0.4416713018796457
DCG score = 2.991795411969586
AP score  = 0.4769026681312379
